# Exercise 3 - Range proofs with ZKPs

Now that we can do a privacy-preserving unlinkable selective disclosure, we can tackle the next step:
removing even more information by only proving the actually relevant information.
For an age check, the verifier doesn't need to know our birthdate.
It's enough if we can convince them that we are above, below, or in the range of a certain age.

## Sections

1. Issuer - Setting up and signing a credential
2. Creating a range proof using LegoGroth16
3. Creating a range proof using Bulletproofs++
4. Discussion: are we anonymous and unlinkable now?
5. Coding exercise - Add an accumlator using a negative membership check

---

## 1. Issuer - Setting up and signing a credential

In [ ]:
// Defining the schema and one credential.

const E_ID_SCHEMA = {
    type: 'object',
    properties: {
        name: { type: 'string' },
        profession: { type: 'string' },
        timeOfBirth: { type: 'integer', minimum: 0},
        height: {type: 'integer', minimum: 130, maximum: 210},
        weight: {type: 'number', minimum: 40, multipleOf: 0.1}
    }
}
const E_ID_DATA = {
    name: "Jack Sparrow",
    timeOfBirth: new Date("1993-08-01T00:00:00").getTime(),
    profession: "IT Manager",
    height: 176,
    weight: 84
}

In [ ]:
// Setting up the issuer

import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

import { BBSKeypair, BBSSignatureParams, BBS_SIGNATURE_PARAMS_LABEL_BYTES,
        CredentialSchema, BBSCredentialBuilder, BBSCredential, SUBJECT_STR } from '@docknetwork/crypto-wasm-ts'

const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

// Generating a keypair once signature parameters are created.
const params = BBSSignatureParams.generate(100, BBS_SIGNATURE_PARAMS_LABEL_BYTES);
const keypair = BBSKeypair.generate(params, stringToBytes('seed1'));

const secretKey = keypair.secretKey;
const publicKey = keypair.publicKey;

// Issing a credential based on the E-ID data based on the specified schema

// default skeleton for the schema
const baseSchema = CredentialSchema.essential();
// we insert our E-ID schema into the data schema position
baseSchema.properties[SUBJECT_STR] = E_ID_SCHEMA;  
let eIDSchema = new CredentialSchema(baseSchema);

const builder = new BBSCredentialBuilder();
builder.schema = eIDSchema;
// assigning data to the schema
builder.subject = E_ID_DATA;

const credential = builder.sign(secretKey);

const credentialToSendToHolder = credential.toJSON();
console.log("This data is sent to the holder:", credentialToSendToHolder);

### Exercises

- Change the content of the credential
- Why is the `timeOfBirth` not a string like "21st of March 1998"?
- Add a new field to the credential, but don't forget to also add it to the schema

---

## 2. Creating a range proof using LegoGroth16

Our first range proof is done using [LegoGroth16](https://eprint.iacr.org/2019/142)

### Issuer

The issuer must create a setup structure with two parts:

- a proving key, used to create the proof, and sent to the holder
- a verifying key, used to verify the proof

In [ ]:
// This section establishes a range proof capability based on the LegoGroth16 algorithm

// The verifier is responsible for creating these keys, then sharing only the proving key with 
// the credential holder
import { BoundCheckSnarkSetup, SetupParam } from '@docknetwork/crypto-wasm-ts';
const provingKey = BoundCheckSnarkSetup();
const snarkProvingKey = provingKey.decompress();
const snarkVerifyingKey = provingKey.getVerifyingKeyUncompressed();

### Credential Holder

With the proving key, the holder can now create a proof.
In our case, the holder reveals two attributes and creates a range proof to prove that: `170 <= height < 190`.

In [ ]:
import { PresentationBuilder } from '@docknetwork/crypto-wasm-ts'
const builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));
// Create a proof that `170 <= height < 190`
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'heightRangeCheck', snarkProvingKey);

const presentation = builder.finalize();
// This is a serialized version (to be sent across an API to the verifier).
const lgProofForVerifier = presentation.toJSON();

console.log("Data sent to verifier:", lgProofForVerifier);

### Sending data to verifier

The following data is now sent to the verifier:

1. Selected data to be revealed
2. Schema of the credential
3. Proof of signature

All this information and more is stored in the "presentation object". Note that this object is a completely serializable JSON object.

### Verifier

The verifier can now make sure that the proof is correct and matches the public key of the issuer. 

In [ ]:
import { Presentation } from '@docknetwork/crypto-wasm-ts'

const predicateParams = new Map([['heightRangeCheck', snarkVerifyingKey]]);

const recreatedPres = Presentation.fromJSON(lgProofForVerifier)

console.log("Credential schema and revealed attributes:", lgProofForVerifier["spec"]["credentials"][0]);
console.log("Verification succeeded:", recreatedPres.verify([publicKey], undefined, predicateParams).verified);

### Exercises

1. What happens if you try to create a proof which is not satisfied by the credential? Why?
2. How does the verifier know the bounds check performed? Do they need this information? Why (not)?
3. Change the proof to be on a different field of the credential
4. Add a second range proof

---

## 3. Creating a range proof using Bulletproofs++

The docknetwork/crypto library offers different proof systems to create the range proofs.
While the LegoGroth16 is based on the well-known and tested Groth16 system, Bulletproofs(++) are somewhat newer, but
as we'll see in the measurement exercise, much faster and smaller.

### Holder

This is very similar to the creation of a proof with LegoGroth16, with two main differences:

- the setup is done on both the holder and the verifier side, using a "Common Reference String"
- these parameters are used instead of the ones created by the verifier in the case of LegoGroth16

In [ ]:
import { BoundCheckBppParams, PresentationBuilder } from '@docknetwork/crypto-wasm-ts'

const builder = new PresentationBuilder();
builder.addCredential(credential, publicKey)
builder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));

// The Holder creates his own parameters for the range proof
const boundCheckBppParams1 = new BoundCheckBppParams(stringToBytes('Common Reference String')).decompress();
builder.enforceBounds(0, 'credentialSubject.height', 170, 190, 'heightRangeCheck', boundCheckBppParams1);

const presentation = builder.finalize();

const bpProofForVerifier = presentation.toJSON()
console.log("Data sent to the verifier:", bpProofForVerifier);

### Verifier

For Bulletproofs++, the verifier needs to recreate the same setup parameters as the prover (holder).
Only the "Common Reference String" must be the same.
This string can be sent from the verifier at the beginning of the 

In [ ]:
import { Presentation } from '@docknetwork/crypto-wasm-ts'

// The verifier creates his own parameters for the range proof
const boundCheckBppParams2 = new BoundCheckBppParams(stringToBytes('Common Reference String')).decompress();

const predicateParams1 = new Map([['heightRangeCheck', boundCheckBppParams2]]);

const recreatedPres = Presentation.fromJSON(bpProofForVerifier);
console.log("Revealed attributes:", recreatedPres.spec.credentials[0].revealedAttributes);
console.log("Verification succeeded:", recreatedPres.verify([publicKey], undefined, predicateParams1).verified);

### Exercises

1. What happens if you try to create a proof which is not satisfied by the credential? Why?
2. How does the verifier know the bounds check performed? Do they need this information? Why (not)?
3. Change the proof to be on a different field of the credential
4. Add a second range proof

---

## 4. Discussion: are we anonymous and unlinkable now?

1. What did we gain by using Zero Knowledge Proofs?
2. What does the verifier learn about our credential?
3. Does this make the proof anonymous? Why? Why not?
4. Is the proof unlinkable? When is it? When isn't it?

---

## 5. Coding exercise - Add an accumlator using a negative membership check

Depending on the time, we propose to do the following exercise: 
implement a simple validity check of the credential using an accumulator.

An accumulator is a cryptographic tool which allows to do (negative) membership proofs.
Instead of simply listing a set of numbers, an accumulator compresses these numbers using cryptographic algorithms.
Different accumulators exist with different trade-ofs:

- the size of the accumulator, depending on the number of elements in it
- whether it's possible to add and remove elements once the accumulator is set up
- if you can prove the existence and/or the absence of an element in the accumulator

Using the docknetwork/crypto library, you can add an inclusion or an absence proof using accumulators.
In order to prove the validity of the credential, the following needs to be done:

1. The issuer holds the accumulator which contains all revoked elements
2. If a credential is revoked, its ID needs to be stored in the accumulator
3. The holder creates an additional proof that their ID is not in the accumulator
4. Now when the verifier checks the proof, they can be sure that the credential of the holder is correct

Some questions:

- What security did we imply with this setup?
- What can go wrong?